In [1]:
import Pkg
Pkg.activate("./")
Pkg.resolve()

  Activating project at `~/Documents/Research/MonitoringMetrology/BackAction.jl/notebooks`
  No Changes to `~/Documents/Research/MonitoringMetrology/BackAction.jl/notebooks/Project.toml`
  No Changes to `~/Documents/Research/MonitoringMetrology/BackAction.jl/notebooks/Manifest.toml`


In [3]:
using Random, Statistics, Plots, LaTeXStrings, DifferentialEquations, LinearAlgebra, BackAction, Base.Threads, Test, ForwardDiff

Precompiling packages...
  12415.6 ms  ✓ BackAction
  1 dependency successfully precompiled in 15 seconds. 287 already precompiled.


In [1]:
using CSV, Tables, DataFrames

In [3]:
using JET
using BenchmarkTools
using Profile, ProfileView

In [7]:
const DELTA_GAMMELMARK::Vector{Float64} = 0.2/0.478 * [0.146,0.262,0.383,0.511,0.648,0.806,0.965,1.118,1.287,1.438,1.596,
               1.755,1.888, 2.057, 2.206, 2.381, 2.539, 2.682, 2.865, 3.013, 3.174, 3.334, 3.452]
const FI_GAMMELMARK::Vector{Float64} = [0.115, 0.221, 0.338, 0.425, 0.507, 0.557, 0.573, 0.566, 0.532, 0.494,
                   0.448, 0.403, 0.363, 0.318, 0.281, 0.241, 0.210, 0.186, 0.156, 0.137, 0.115, 0.101, 0.09348];

In [4]:
const NCHANNELS0::Int64 = 1
#const NCHANNELS::Int64 = 1
const NLEVELS::Int64 = 2
const OMEGA = 1.0
const GAMMA = 0.5

0.5

In [4]:
# Set up the unravelings
function sample_c2(seed)
    theta = pi*rand()
    phi = 2*pi*rand()
    return [cos(theta*0.5); exp(1im*phi)*sin(theta*0.5)] 
end
function get_random_unraveling(seed)
    Random.seed!(seed)
    T = sample_c2(seed) 
    T = reshape(T/norm(T), 2,1)
    T_curvy = [real(T[1]) imag(T[1]); real(T[2]) imag(T[2]) ]
    F = svd(T_curvy) # Factorization object
    P = F.V * Diagonal(F.S) * F.Vt
    O = F.U * F.Vt
    T_curvy_dyne = O' * T_curvy
    T_dyne = O' * real.(T) + 1im * O' * imag.(T)
    Ta = T 
    Tb = T_dyne 
    Tc = T_dyne 
    Ts = Dict([("a", Ta), ("b", Tb), ("c", Tc)]) # Dictionary with the Ts
    alpha0 = [0.5 + 0im; 0.5 + 0im] 
    alphas = Dict([("a", alpha0), ("b", alpha0), ("c", O' * alpha0)]) # Dictionary with the alphas

    return Ts, alphas
end


get_random_unraveling (generic function with 1 method)

In [9]:
# Base parametric operators

L0 = d -> sqrt(GAMMA) * BackAction.sigma_m
H0 = d -> d*[[0, 0] [0, 1.0+0im]]  + 0.5*OMEGA*BackAction.sigma_x 

# Unraveling parametrix operators
function obtain_parametric_unraveling_operators(T::Matrix{T1}, alpha::Vector{T1}) where {T1<:Complex}
    # First do the unitary mixing 
    Ls_mixed = BackAction.isometric_mixing([L0], T, NLEVELS, size(T)[1])
    H = BackAction.add_cfield_hamiltonian_correctionterm(H0, Ls_mixed, alpha, NLEVELS)
    Ls_mixed_cfield = BackAction.add_cfields(Ls_mixed, alpha, NLEVELS)
    He = BackAction.get_Heff(H, Ls_mixed_cfield, NLEVELS)
    return Ls_mixed_cfield, H, He
end


obtain_parametric_unraveling_operators (generic function with 1 method)

In [10]:
function evaluate_and_fill_Ls!(Ls::Vector{TJ}, theta::T2,
        Ls_store::Array{T1}, nchannels::T3) where {T1<:Complex, T2<:Real, T3<:Int,TJ<:Function} 
    for k in nchannels
       Ls_store[:,:, k] .= Ls[k](theta) 
    end 
end 

evaluate_and_fill_Ls! (generic function with 1 method)

In [12]:
#################################### SYSTEM DEFINITION 
# 0. Choose the parameters
delta = DELTA_GAMMELMARK[1]
params_simul = SimulParameters([0.0+0im, 1.0],
    300.0, # Final time. Set very long so that all trajectories jump
    1, # seed
    1_000, # Number of trajectories
    # GILLIPSIE STUFF 
    75_000, 3.0, 1e-3 )
tspan = (0.0, params_simul.tf)
# 1. define the T and alpha
T = reshape([1.0+0im], 1,1) # MUST BE CONSISTENT WITH THE DIMENSIONS OF ALPHA
alpha = [0.0+0im]
nchannels = length(alpha)
# 2. Obtain the parametrizations of Ls, H and He
Ls_par, H_par, He_par = obtain_parametric_unraveling_operators(T, alpha)
# 3. Evaluate the parametrizations
Ls = Array{ComplexF64}(undef, NLEVELS, NLEVELS, nchannels)
evaluate_and_fill_Ls!(Ls_par, delta, Ls, nchannels)
## 4. Initialzie the system
sys = System(H_par(delta), Ls, NLEVELS, nchannels)

System(NLEVELS=2
NCHANNELS=1
H=ComplexF64[0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.06108786610878661 + 0.0im]
Ls=ComplexF64[0.0 + 0.0im 0.7071067811865476 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im;;;]
J=ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.5000000000000001 + 0.0im])
Heff=ComplexF64[0.0 + 0.0im 0.5 + 0.0im; 0.5 + 0.0im 0.06108786610878661 - 0.25000000000000006im])

In [14]:
# Set up the derivatives of the jump operators
dLs = Array{ComplexF64}(undef, NLEVELS, NLEVELS, nchannels)
for k in 1:nchannels
    ForwardDiff.derivative!(view(dLs, :, :, k), Ls_par[k], GAMMA)
end
# Information for the derivatives
dtheta = delta/1000
He_forward = He_par(delta + 0.5*dtheta )
He_backward = He_par(delta - 0.5*dtheta)

2×2 Matrix{ComplexF64}:
 0.0+0.0im        0.5+0.0im
 0.5+0.0im  0.0610573-0.25im

In [16]:
@time begin
    sim = get_sol_jumps_monitoring(dtheta, dLs, He_forward, He_backward, sys, params_simul, tspan; 
                                    save_on=false, save_start=false, save_end=true);
end

  2.907797 seconds (40.68 M allocations: 1.421 GiB, 7.92% gc time)


EnsembleSolution Solution of length 1000 with uType:
ODESolution{ComplexF64, 2, Vector{Vector{ComplexF64}}, Nothing, Nothing, Vector{Float64}, Vector{Vector{Vector{ComplexF64}}}, Nothing, ODEProblem{Vector{ComplexF64}, Tuple{Float64, Float64}, true, SciMLBase.NullParameters, ODEFunction{true, SciMLBase.AutoSpecialize, FunctionWrappersWrappers.FunctionWrappersWrapper{Tuple{FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{ComplexF64}, Vector{ComplexF64}, SciMLBase.NullParameters, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{Complex{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, ComplexF64}, ComplexF64, 1}}}, Vector{Complex{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, ComplexF64}, ComplexF64, 1}}}, SciMLBase.NullParameters, Float64}}, FunctionWrappers.FunctionWrapper{Nothing, Tuple{Vector{Complex{ForwardDiff.Dual{ForwardDiff.Tag{DiffEqBase.OrdinaryDiffEqTag, ComplexF64}, ComplexF64, 1}}}, Vector{ComplexF64}, SciMLBase.NullParame

In [17]:
function calculate_fi(ensol)
    aux = 0.0
    for sol in ensol 
        psi = sol.u[end]/norm(sol.u[end])
        phi = sol.prob.kwargs[:callback].affect!.cache_phi
        aux = aux + (2*real(dot(psi, phi)))^2
    end 
    return aux/length(ensol)
end 

calculate_fi (generic function with 1 method)

In [18]:
calculate_fi(sim)

33.022511381598136

In [20]:
FI_GAMMELMARK[1]*300

34.5